#RO Project#

1. Read instances' data from file
2. Initialize the parameters read from file
3. n = number of customers
4. c = matrix of costs
5. ml = max load of the vehicle
6. l = linehaul values vector
7. b = backhaul values vector
8. Initialize the matrix s of savings 
9. for i = 1:n
    1. for j = 1:n
        1. If i == j
            1. s_ij = +inf
        2. else
            1. s_ij = c_i0 + c_0j - c_ij
10. Routes = list of empty routes
11. for i = 1:n
    1. Routes_i = (0,i,0)
12. Order the savings s in a non increasing fashion
13. for each route in Routes:
    1. while this route can be feasibly (respecting the constraints) merged further:
	   1.determine the first saving s_ki or s_jl in the ordered list that can feasibly be used to merge the current route with another route containing either the arc (k, 0) or (0, l). 
       	   2. merge the two routes.



In [194]:
import numpy as np
import sys

In [195]:
filename = 'Instances/N1.txt'
with open(filename, 'r') as f:
    content = f.read()
    print(content)

150
1
11
12000   16000   0   5700
1725   21949   0   820   0
22443   15651   0   291   0
11622   7533   0   344   0
14613   7181   0   244   0
771   10569   0   714   0
18508   15099   0   311   0
16029   5701   0   267   0
20666   29528   0   638   0
7898   29590   0   362   0
7557   11209   0   594   0
22533   27056   0   548   0
13861   10795   0   307   0
10578   3881   0   382   0
1625   19201   0   335   0
18608   26115   0   394   0
5670   26554   0   631   0
21409   9013   0   755   0
12465   23593   0   341   0
16852   18097   0   206   0
8410   1322   0   417   0
3863   29503   0   693   0
4114   9954   0   606   0
4736   18223   0   577   0
9835   10318   0   268   0
18257   20218   0   396   0
12606   23154   0   738   0
18578   11360   0   709   0
22262   8762   0   577   0
4626   26730   0   264   0
1976   18060   0   334   0
22413   12376   0   410   0
9725   11171   0   352   0
22724   3555   0   292   0
8260   9540   0   589   0
3994   6140   0   268   0
7979   3116   

In [196]:
content = content.split('\n')[0:-1] #-1 because the last element is a newline, so we skip it

In [197]:
n_of_customers = int(content[0])
n_of_vehicles = int(content[2])
capacity = int(content[3].split()[3])
linehaul_vector = [0] + [int(i.split()[2]) for i in content[4:]] #linehaul data is in the fourth column
backhaul_vector = [0] + [int(i.split()[3]) for i in content[4:]] #and backhaul is in the third

In [198]:
import math

def euclidean_distance2D(point1, point2):
    
    x1, y1 = point1
    x2, y2 = point2
    
    return (math.sqrt((x1-x2)**2 + (y1-y2)**2))

In [199]:
cost_matrix = []

x_coords = [int(i.split()[0]) for i in content[3:]]
y_coords = [int(i.split()[1]) for i in content[3:]]

for i, (x1, y1) in enumerate(zip(x_coords, y_coords)): #could also enumerate over y_coords, the dimensions are equal
    
    current_row = []
    
    for j, (x2, y2) in enumerate(zip(x_coords, y_coords)):
        
        current_row += [euclidean_distance2D((x1,y1), (x2,y2))]
        
    cost_matrix += [current_row]
        
        

In [200]:
cost_matrix = np.array(cost_matrix)
savings_matrix = np.zeros((n_of_customers + 1, n_of_customers + 1))

In [201]:
for i in range(n_of_customers + 1):
    for j in range(n_of_customers + 1):
        if (i==j):
            savings_matrix[i][j] = sys.maxsize
        else:
            savings_matrix[i][j] = cost_matrix[i][0] + cost_matrix[0][j] - cost_matrix[i][j]

In [202]:
routes = []

In [203]:
for i in range(n_of_customers):
    
    routes += [(0, i+1, 0)]

In [204]:
ordered_savings = []

for (i, j), el in np.ndenumerate(savings_matrix):
    ordered_savings += [(el, (i, j))]

ordered_savings.sort(key=lambda x: x[0], reverse=True)
ordered_i_savings = list(ordered_savings)
ordered_j_savings = list(ordered_savings)
ordered_i_savings.sort(key=lambda x: (x[1][0], -x[0]))
ordered_j_savings.sort(key=lambda x: (x[1][1], -x[0]))


In [205]:
def is_route_valid (route_to_check):
    
    is_route_possible = True
    cost_of_possible_route_linehaul = 0
    cost_of_possible_route_backhaul = 0
    i = 0
    
    for node_touched in route_to_check[1:-1]:
        
        if linehaul_vector[node_touched] == 0:#linehaul_vector is 0 if a node is not a linehaul node, so we have to go to the next for
            break
        else:
            cost_of_possible_route_linehaul += linehaul_vector[node_touched]
            i+=1
            
    starting_point = i
    if i < len(route_to_check[1:-1]):
        for node_touched in route_to_check[starting_point+1:-1]:
            
            if backhaul_vector[node_touched] == 0 and i != len(route_to_check): #we are trying to linehaul AFTER we started backhauling... constraint broken:
                is_route_possible = False
                #print(i, backhaul_vector[node_touched], node_touched)
            else:
                cost_of_possible_route_backhaul += backhaul_vector[node_touched]
                i+=1
                
        
    if (cost_of_possible_route_backhaul > capacity or cost_of_possible_route_linehaul > capacity):
        is_route_possible = False
        
    if (cost_of_possible_route_linehaul == 0): # we are only backhauling
        is_route_possible = False
        
        
    return is_route_possible

    

In [206]:
def is_route_merge_valid(id_route1, id_route2):
    
    """First, try if you can merge it with the end of route1 to end of route2"""
        
    possible_route = list( routes[id_route1][:-1] + routes[id_route2][1:]) #the merge itself is just skipping the left and right
    
    is_right_possible = is_route_valid(possible_route)
    """Now the other possible route."""
        
    possible_route = list( routes[id_route2][:-1] + routes[id_route1][1:]) #the merge itself is just skipping the left and right
    
    is_left_possible = is_route_valid(possible_route)
    
    #print(is_left_possible, is_right_possible)
        
    if not is_left_possible and not is_right_possible:
        return ('')
    elif is_left_possible and is_right_possible:
        return ('lr')
    elif is_left_possible:
        return 'l'
    else:
        return 'r'
        
                


In [207]:
def find_first_merge(id_curr_route, available_merges):
    
    
    for saving in ordered_i_savings:
        
        if saving[1][0] > id_curr_route+1:
            break
        
        if(saving[1][0] == id_curr_route+1 and saving[0] != sys.maxsize):
            
            for merge in available_merges:
                if saving[1][1] == merge[0]+1:
                    return merge
        
    for saving in ordered_j_savings:
        
        if saving[1][0] > id_curr_route+1:
            break
            
        if(saving[1][0] == id_curr_route and saving[0] != sys.maxsize):
            
            for merge in available_merges:

                if saving[1][1] == merge[0]+1:
                    return merge
    

In [208]:
def merge_two_routes(id_route, found_id, direction_of_merge):

    if id_route== found_id:
        return
    
    tmp_list1 = list(routes[id_route])
    tmp_list2 = list(routes[found_id])
        
    tmp_list1.pop(0)
    tmp_list1.pop(-1)
    tmp_list2.pop(0)
    tmp_list2.pop(-1)
    
    if direction_of_merge == 'l':
        
        routes[id_route] = tuple([0]+tmp_list2+tmp_list1+[0])
    
    if direction_of_merge == 'r':
        
        routes[id_route] = tuple([0]+tmp_list1+tmp_list2+[0])
    
    routes[found_id] = (-1,-1,-1)


    

In [209]:
direction_of_merge = is_route_merge_valid(10, 18)
print(direction_of_merge)

In [210]:
for id_route1, route in enumerate(routes):
    
    while True:
                
        if (routes[id_route1] == (-1,-1,-1)):
            break
        
        """
        Next thing to do should be modify the for underneath this comment so that it saves an array of valid routes, and at the end, if the array is not empty,
        check which one of the possible merges is the best and then merge with that
        
        """
                
        found_id = -1
        direction_of_merge = 'l'
        
        available_merges = []
        for id_route2, route2 in enumerate(routes):

            if (routes[id_route2] == (-1,-1,-1)):
                continue
               
            if (id_route1 != id_route2):
                    direction_of_merge = is_route_merge_valid(id_route1, id_route2)
                    if direction_of_merge != "":
                            if direction_of_merge == 'l' or direction_of_merge == 'r':
                                available_merges += [(id_route2, direction_of_merge)]
                            else:
                                available_merges += [(id_route2, 'l')]
                                available_merges += [(id_route2, 'r')]
                    
        
        if (len(available_merges) == 0):
            break
        #print(available_merges)
        found_id, direction_of_merge= find_first_merge(id_route1, available_merges)
        if routes[found_id] != (-1,-1,-1):
            merge_two_routes(id_route1, found_id, direction_of_merge)
                 

In [211]:
routes

[(0,
  130,
  138,
  90,
  122,
  106,
  68,
  53,
  85,
  86,
  137,
  88,
  1,
  14,
  21,
  45,
  29,
  40,
  30,
  16,
  9,
  23,
  46,
  5,
  36,
  0),
 (0,
  72,
  56,
  95,
  103,
  123,
  78,
  134,
  69,
  114,
  133,
  92,
  2,
  42,
  31,
  39,
  28,
  17,
  33,
  11,
  38,
  43,
  19,
  0),
 (0,
  55,
  116,
  144,
  64,
  79,
  129,
  150,
  146,
  58,
  99,
  147,
  82,
  3,
  13,
  50,
  20,
  47,
  7,
  4,
  49,
  48,
  35,
  34,
  24,
  22,
  12,
  0),
 (-1, -1, -1),
 (-1, -1, -1),
 (0,
  101,
  60,
  100,
  131,
  76,
  102,
  51,
  91,
  107,
  74,
  65,
  73,
  6,
  27,
  25,
  8,
  15,
  37,
  26,
  18,
  44,
  32,
  10,
  0),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),
 (-1, -1, -1),

In [212]:
for route in routes:
    if route == (-1,-1,-1):
        continue
    total_l = 0
    total_b = 0
    for el in route:
        if el == 0:
            continue
        
        total_l += linehaul_vector[el]
        total_b += backhaul_vector[el]
    print(route)
    print(total_l)
    print(total_b)

(0, 130, 138, 90, 122, 106, 68, 53, 85, 86, 137, 88, 1, 14, 21, 45, 29, 40, 30, 16, 9, 23, 46, 5, 36, 0)
5652
5691
(0, 72, 56, 95, 103, 123, 78, 134, 69, 114, 133, 92, 2, 42, 31, 39, 28, 17, 33, 11, 38, 43, 19, 0)
5617
5649
(0, 55, 116, 144, 64, 79, 129, 150, 146, 58, 99, 147, 82, 3, 13, 50, 20, 47, 7, 4, 49, 48, 35, 34, 24, 22, 12, 0)
5698
5659
(0, 101, 60, 100, 131, 76, 102, 51, 91, 107, 74, 65, 73, 6, 27, 25, 8, 15, 37, 26, 18, 44, 32, 10, 0)
5648
5406
(0, 148, 81, 94, 135, 111, 71, 112, 80, 41, 0)
5634
434
(0, 121, 93, 98, 66, 75, 67, 125, 141, 145, 52, 0)
5512
0
(0, 128, 57, 59, 61, 118, 77, 108, 115, 109, 54, 0)
5620
0
(0, 89, 104, 119, 63, 70, 142, 83, 96, 62, 0)
5647
0
(0, 120, 117, 149, 124, 97, 143, 136, 110, 127, 84, 0)
5692
0
(0, 105, 132, 140, 113, 139, 126, 87, 0)
3380
0


In [213]:
for route in routes:
    
    
    if route != (-1, -1, -1):
        
        print ('Route:')
        print (route)
        
        for item in route[1:-1]:
            
            if linehaul_vector[item] != 0:
                print('L')
                
            if backhaul_vector[item] != 0:
                print('B')
                


Route:
(0, 130, 138, 90, 122, 106, 68, 53, 85, 86, 137, 88, 1, 14, 21, 45, 29, 40, 30, 16, 9, 23, 46, 5, 36, 0)
L
L
L
L
L
L
L
L
L
L
L
B
B
B
B
B
B
B
B
B
B
B
B
B
Route:
(0, 72, 56, 95, 103, 123, 78, 134, 69, 114, 133, 92, 2, 42, 31, 39, 28, 17, 33, 11, 38, 43, 19, 0)
L
L
L
L
L
L
L
L
L
L
L
B
B
B
B
B
B
B
B
B
B
B
Route:
(0, 55, 116, 144, 64, 79, 129, 150, 146, 58, 99, 147, 82, 3, 13, 50, 20, 47, 7, 4, 49, 48, 35, 34, 24, 22, 12, 0)
L
L
L
L
L
L
L
L
L
L
L
L
B
B
B
B
B
B
B
B
B
B
B
B
B
B
Route:
(0, 101, 60, 100, 131, 76, 102, 51, 91, 107, 74, 65, 73, 6, 27, 25, 8, 15, 37, 26, 18, 44, 32, 10, 0)
L
L
L
L
L
L
L
L
L
L
L
L
B
B
B
B
B
B
B
B
B
B
B
Route:
(0, 148, 81, 94, 135, 111, 71, 112, 80, 41, 0)
L
L
L
L
L
L
L
L
B
Route:
(0, 121, 93, 98, 66, 75, 67, 125, 141, 145, 52, 0)
L
L
L
L
L
L
L
L
L
L
Route:
(0, 128, 57, 59, 61, 118, 77, 108, 115, 109, 54, 0)
L
L
L
L
L
L
L
L
L
L
Route:
(0, 89, 104, 119, 63, 70, 142, 83, 96, 62, 0)
L
L
L
L
L
L
L
L
L
Route:
(0, 120, 117, 149, 124, 97, 143, 136, 110, 127, 84, 0)


In [214]:
total_cost = 0

for route in routes:
    
    if route != (-1, -1, -1):
        
        for idx, item in enumerate(route[:-1]):
            
            total_cost+=cost_matrix[item, route[idx+1]]

In [215]:
total_cost

956302.2721974939

In [216]:
new_routes = []

for route in routes:
    
    if route != (-1, -1, -1):
        
        new_routes += [route]
        
routes = new_routes

In [217]:
def find_best_improvement():
    
    possible_changes = []
    
    for id_route1, route1 in enumerate(routes):
        
        for id_item_to_move, item_to_move in enumerate(route1[1:-1]): # remember that id is actually +1 if considered in the full route
            
            for id_route2, route2 in enumerate(routes):
                #print('route1:', route1)
                #print('item_to_move:', item_to_move)
                #print('route2:', route2)
                
                if (id_route1 == id_route2):
                    temp_route = list(route1[:id_item_to_move+1] + route1[id_item_to_move+2:])
                else:
                    temp_route = route2
                
                for id_moved, item in enumerate(temp_route[1:]): # just as above, careful with id
                    possible_change = list([0] + list(temp_route[1:id_moved+1]) + [item_to_move] + list(temp_route[id_moved+1:]))
                    if (is_route_valid(possible_change)): #and is_route_valid (list(route1[:id_item_to_move+1] + route1[id_item_to_move+2:]))):
                        #print(possible_change)
                        #print(list([0] + list(temp_route[1:id_moved+1]) + [item_to_move] + list(temp_route[id_moved+1:])))
                        #print(id_route1, id_route2, id_item_to_move, id_moved)
                        possible_changes += [(possible_change, item_to_move, id_route1, id_route2, id_item_to_move, id_moved)]
                        
                        
    most_improvement = 0
    best_change = ()

    for id_change, change in enumerate(possible_changes):

        cost_of_old_routes = 0
        cost_of_new_routes = 0
        (possible_change, item_to_move, id_route1, id_route2, id_item_to_move, id_moved) = change
        
        if (id_route1 == id_route2):
            
            for idx, item in enumerate(routes[id_route1][:-1]):
                
                cost_of_old_routes+=cost_matrix[item, routes[id_route1][idx+1]]

            new_route_1 = []
            new_route_2 = possible_change

            for idx, item in enumerate(new_route_2[:-1]):

                cost_of_new_routes+=cost_matrix[item, new_route_2[idx+1]]
        
        else:
            
        
            for idx, item in enumerate(routes[id_route1][:-1]):

                cost_of_old_routes+=cost_matrix[item, routes[id_route1][idx+1]]

            for idx, item in enumerate(routes[id_route2][:-1]):

                cost_of_old_routes+=cost_matrix[item, routes[id_route2][idx+1]]


            new_route_1 = list(routes[id_route1][:id_item_to_move+1] + routes[id_route1][id_item_to_move+2:])
            new_route_2 = possible_change

            for idx, item in enumerate(new_route_1[:-1]):

                cost_of_new_routes+=cost_matrix[item, new_route_1[idx+1]]

            for idx, item in enumerate(new_route_2[:-1]):

                cost_of_new_routes+=cost_matrix[item, new_route_2[idx+1]]

        #print('cost of old routes:', cost_of_old_routes)
        #print('cost of new routes:', cost_of_new_routes)
        
        if (cost_of_new_routes < cost_of_old_routes):
            #print(cost_of_old_routes - cost_of_new_routes)

            if cost_of_old_routes - cost_of_new_routes > most_improvement:
                most_improvement = cost_of_old_routes - cost_of_new_routes
                best_change = ( most_improvement, (id_route1, new_route_1) , (id_route2, new_route_2) )
                #print(most_improvement)
                
                
    
    #print(most_improvement)
    return best_change
            
        

In [218]:
new_cost = 0
previous_cost = total_cost
threshold = 5

while previous_cost - new_cost > threshold:
    
    possible_change = find_best_improvement()
    previous_cost = new_cost

    if possible_change != ():

        improvement, (old_route1_index, new_route_1), (old_route2_index, new_route_2) = possible_change

        print('modifying:', old_route1_index, old_route2_index)
        print('from:', routes[old_route1_index], ' and:', routes[old_route2_index])
        print('to:', new_route_1, 'and: ', new_route_2)

        routes[old_route1_index] = new_route_1
        routes[old_route2_index] = new_route_2

        new_cost = previous_cost - improvement


modifying: 3 3
from: (0, 101, 60, 100, 131, 76, 102, 51, 91, 107, 74, 65, 73, 6, 27, 25, 8, 15, 37, 26, 18, 44, 32, 10, 0)  and: (0, 101, 60, 100, 131, 76, 102, 51, 91, 107, 74, 65, 73, 6, 27, 25, 8, 15, 37, 26, 18, 44, 32, 10, 0)
to: [] and:  [0, 101, 60, 76, 100, 131, 102, 51, 91, 107, 74, 65, 73, 6, 27, 25, 8, 15, 37, 26, 18, 44, 32, 10, 0]
modifying: 2 2
from: (0, 55, 116, 144, 64, 79, 129, 150, 146, 58, 99, 147, 82, 3, 13, 50, 20, 47, 7, 4, 49, 48, 35, 34, 24, 22, 12, 0)  and: (0, 55, 116, 144, 64, 79, 129, 150, 146, 58, 99, 147, 82, 3, 13, 50, 20, 47, 7, 4, 49, 48, 35, 34, 24, 22, 12, 0)
to: [] and:  [0, 55, 116, 144, 64, 79, 129, 150, 146, 58, 99, 147, 82, 3, 13, 50, 20, 47, 48, 7, 4, 49, 35, 34, 24, 22, 12, 0]
modifying: 0 5
from: (0, 130, 138, 90, 122, 106, 68, 53, 85, 86, 137, 88, 1, 14, 21, 45, 29, 40, 30, 16, 9, 23, 46, 5, 36, 0)  and: (0, 121, 93, 98, 66, 75, 67, 125, 141, 145, 52, 0)
to: [0, 138, 90, 122, 106, 68, 53, 85, 86, 137, 88, 1, 14, 21, 45, 29, 40, 30, 16, 9, 23,

In [219]:
improvement

4.980283068303834

In [220]:
total_cost = 0

for route in routes:
    
    if route != (-1, -1, -1):
        
        for idx, item in enumerate(route[:-1]):
            
            total_cost+=cost_matrix[item, route[idx+1]]
print(total_cost)

493934.7260015929


In [ ]:
for route in routes:
    if route == (-1,-1,-1):
        continue
    total_l = 0
    total_b = 0
    for el in route:
        if el == 0:
            continue
        
        total_l += linehaul_vector[el]
        total_b += backhaul_vector[el]
    print(route)
    print(total_l)
    print(total_b)

In [306]:
for route in routes:
    
    
    if route != (-1, -1, -1):
        
        print ('Route:')
        print (route)
        
        for item in route[1:-1]:
            
            if linehaul_vector[item] != 0:
                print('L')
                
            if backhaul_vector[item] != 0:
                print('B')


Route:
[0, 23, 23, 5, 5, 5, 5, 2, 0]
L
L
B
B
B
B
B
Route:
[0, 10, 10, 4, 4, 4, 0]
L
L
B
B
B
Route:
[0, 6, 6, 6, 0]
L
L
L
Route:
[0, 8, 8, 0]
L
L
Route:
[0, 20, 20, 9, 0]
L
L
L
Route:
[0, 11, 11, 0]
L
L
Route:
[0, 21, 21, 16, 0]
L
L
L


In [193]:
improvement

4.391110106837004

In [192]:
total_cost = 0

for route in routes:
    
    if route != (-1, -1, -1):
        
        for idx, item in enumerate(route[:-1]):
            
            total_cost+=cost_matrix[item, route[idx+1]]
print(total_cost)

625134.3234476394


In [188]:
def find_best_exchange():
    
    possible_changes = []
    
    for id_route1, route1 in enumerate(routes):
        
        for id_item_to_exchange1, item_to_exchange1 in enumerate(route1[1:-1]): # remember that id is actually +1 if considered in the full route
            
            for id_route2, route2 in enumerate(routes):
                
                if(id_route1 > id_route2):
                    continue #maybe?
                
                    

                    
                
                for id_item_to_exchange2, item_to_exchange2 in enumerate(route2[1:-1]):
                    
                    possible_change1 = list(route1)
                    possible_change2 = list(route2)
                    
                    if id_route1 == id_route2:
                        possible_change1[id_item_to_exchange1], possible_change1[id_item_to_exchange2] = possible_change1[id_item_to_exchange2], possible_change1[id_item_to_exchange1]
                        possible_change2[id_item_to_exchange1], possible_change2[id_item_to_exchange2] = possible_change2[id_item_to_exchange2], possible_change2[id_item_to_exchange1]

                    
                    else:
                        possible_change1[id_item_to_exchange1+1] = item_to_exchange2
                        possible_change2[id_item_to_exchange2+1] = item_to_exchange1
                    #print((possible_change1, possible_change2))


                
                    if (is_route_valid(possible_change1) and is_route_valid(possible_change2)): #and is_route_valid (list(route1[:id_item_to_move+1] + route1[id_item_to_move+2:]))):


                        #print(possible_change)
                        #print(list([0] + list(temp_route[1:id_moved+1]) + [item_to_move] + list(temp_route[id_moved+1:])))
                        #print(id_route1, id_route2, id_item_to_move, id_moved)
                        possible_changes += [(possible_change1, possible_change2, item_to_exchange1, item_to_exchange2, id_route1, id_route2, id_item_to_exchange1, id_item_to_exchange2)]

                        
    most_improvement = 0
    best_change = ()

    for id_change, change in enumerate(possible_changes):

        cost_of_old_routes = 0
        cost_of_new_routes = 0
        (possible_change1, possible_change2, item_to_exchange1, item_to_exchange2, id_route1, id_route2, id_item_to_exchange1, id_item_to_exchange2) = change
        
        if (id_route1 == id_route2):
            
            for idx, item in enumerate(routes[id_route1][:-1]):
                
                cost_of_old_routes+=cost_matrix[item, routes[id_route1][idx+1]]

            new_route_1 = []
            new_route_2 = possible_change2

            for idx, item in enumerate(new_route_2[:-1]):

                cost_of_new_routes+=cost_matrix[item, new_route_2[idx+1]]
        
        else:
            
        
            for idx, item in enumerate(routes[id_route1][:-1]):

                cost_of_old_routes+=cost_matrix[item, routes[id_route1][idx+1]]

            for idx, item in enumerate(routes[id_route2][:-1]):

                cost_of_old_routes+=cost_matrix[item, routes[id_route2][idx+1]]


            new_route_1 = possible_change1
            new_route_2 = possible_change2

            for idx, item in enumerate(new_route_1[:-1]):

                cost_of_new_routes+=cost_matrix[item, new_route_1[idx+1]]

            for idx, item in enumerate(new_route_2[:-1]):

                cost_of_new_routes+=cost_matrix[item, new_route_2[idx+1]]

        #print('cost of old routes:', cost_of_old_routes)
        #print('cost of new routes:', cost_of_new_routes)
        
        if (cost_of_new_routes < cost_of_old_routes):
            #print(cost_of_old_routes - cost_of_new_routes)

            if cost_of_old_routes - cost_of_new_routes > most_improvement:
                most_improvement = cost_of_old_routes - cost_of_new_routes
                best_change = ( most_improvement, (id_route1, new_route_1) , (id_route2, new_route_2) )
                #print(most_improvement)
                
                
    
    #print(most_improvement)
    return best_change

In [189]:
new_cost = 0
previous_cost = total_cost
threshold = 5

while previous_cost - new_cost > threshold:
    
    possible_change = find_best_exchange()
    previous_cost = new_cost

    if possible_change != ():

        improvement, (old_route1_index, new_route_1), (old_route2_index, new_route_2) = possible_change

        print('modifying:', old_route1_index, old_route2_index)
        print('from:', routes[old_route1_index], ' and:', routes[old_route2_index])
        print('to:', new_route_1, 'and: ', new_route_2)

        routes[old_route1_index] = new_route_1
        routes[old_route2_index] = new_route_2

        new_cost = previous_cost - improvement

modifying: 5 5
from: (0, 101, 60, 100, 131, 76, 102, 51, 91, 107, 74, 65, 73, 6, 27, 25, 8, 15, 37, 26, 18, 44, 32, 10, 0)  and: (0, 101, 60, 100, 131, 76, 102, 51, 91, 107, 74, 65, 73, 6, 27, 25, 8, 15, 37, 26, 18, 44, 32, 10, 0)
to: [] and:  [0, 101, 60, 100, 76, 131, 102, 51, 91, 107, 74, 65, 73, 6, 27, 25, 8, 15, 37, 26, 18, 44, 32, 10, 0]
modifying: 40 51
from: (0, 148, 81, 94, 135, 111, 71, 112, 80, 41, 0)  and: (0, 121, 93, 98, 66, 75, 67, 125, 141, 145, 52, 0)
to: [0, 148, 121, 94, 135, 111, 71, 112, 80, 41, 0] and:  [0, 81, 93, 98, 66, 75, 67, 125, 141, 145, 52, 0]
modifying: 1 1
from: (0, 72, 56, 95, 103, 123, 78, 134, 69, 114, 133, 92, 2, 42, 31, 39, 28, 17, 33, 11, 38, 43, 19, 0)  and: (0, 72, 56, 95, 103, 123, 78, 134, 69, 114, 133, 92, 2, 42, 31, 39, 28, 17, 33, 11, 38, 43, 19, 0)
to: [] and:  [0, 72, 56, 95, 103, 123, 78, 134, 69, 114, 133, 92, 2, 42, 31, 39, 28, 17, 33, 43, 38, 11, 19, 0]
modifying: 0 0
from: (0, 130, 138, 90, 122, 106, 68, 53, 85, 86, 137, 88, 1, 14, 2